### Week 10 - text style transfer

Hello, sitzen class A.412C!

Based on your browser search history, we conclude that you have an above average skill in natural language processing. In our benevolence, we give you a chance to contribute your skills to upholding the happiest society in the universe. Are you up to the task?

As you know, our most recent breakthrough was replacing 97% restaurant workers with BFGHQBERT+++ autonomous food dispensers.

Yet a some radical elements failed to recognize the greater good that we brought them. They mistakenly voice their ignorant opinions about our new INGSOC-approved restaurants, brining dangerous doubt to the minds of our loyal citzens.

Surely you cannot tolerate such infidelity! Our loyal citzens demand that you rectify their mistake. _You must build a model that will automatically improve their ignorant thoughts and replace them with the thoughts they should actually have._

Attached below are the INGSOC-approved datasets for ignorant and correct thoughts. The scientific terminology is for wrong opinions and correct opinions is "negative" and "positive", respectively.

Respond within 7 days or you will lose 3.7629 citzenship points.

![img](https://ih1.redbubble.net/image.1254830934.9884/poster,504x498,f8f8f8-pad,400x240,f8f8f8.jpg)

In [1]:
!pip install -q transformers
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.train.0 -O train_negative
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.train.1 -O train_positive
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.dev.0 -O dev_negative
!wget -q https://github.com/shentianxiao/language-style-transfer/raw/master/data/yelp/sentiment.dev.1 -O dev_positive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 28.7 MB/s eta 0:00:00


In [2]:
!head -n 5 ./dev_positive
!echo
!head -n 5 ./dev_negative

staff behind the deli counter were super nice and efficient !
love this place !
the staff are always very nice and helpful .
the new yorker was amazing .
very ny style italian deli .

ok never going back to this place again .
easter day nothing open , heard about this place figured it would ok .
the host that walked us to the table and left without a word .
it just gets worse .
the food tasted awful .


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'

if device == 'cpu':
    print("Fine-tuning BERT without an accelerator is not party-approved.")

### Part 1: Masked language model

Attached below you can find the INGSOC-compliant training code that fine-tunes a BERT model for Masked Language Modeling.

You shall use this model to generate positive replacements for negative tokens.

In [4]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_mlm_positive = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True).to(device).train(True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

print("Preparing the training data...")
dataset = LineByLineTextDataset(
    file_path="./train_positive", tokenizer=tokenizer, block_size=128)

print("Dataset ready!")

trainer = Trainer(
    model=bert_mlm_positive, train_dataset=dataset, 
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.15),
    args=TrainingArguments(
        output_dir="./bert_mlm_positive", overwrite_output_dir=True,
        num_train_epochs=1, per_device_train_batch_size=32,
        save_steps=10_000, save_total_limit=2),
)

trainer.train()

Preparing the training data...


/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Dataset ready!


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,2.360500
1000,2.154000
1500,2.084700
2000,2.005100
2500,1.988100
3000,1.928600
3500,1.950500
4000,1.898600
4500,1.865300
5000,1.881400


TrainOutput(global_step=8354, training_loss=1.921871259276993, metrics={'train_runtime': 1313.2719, 'train_samples_per_second': 203.548, 'train_steps_per_second': 6.361, 'total_flos': 2570714209611000.0, 'train_loss': 1.921871259276993, 'epoch': 1.0})

In [ ]:
# <Build and train a MLM for incorrect opinions>

bert_mlm_negative = <...>

<A whole lot of your code>

### Part 2: Replace tokens

You can now use the two masked language models to align user opinions. You can do so with the following steps:

1. Find tokens where the ratio $(P_{positive}(x) + \epsilon) / (P_{negative}(x) + \epsilon)$ is the smallest
2. Replace those tokens with one of $k$ most likely tokens according to $P_{positive}(x)$.
3. Rinse, repeat

You can find the full procedure at https://arxiv.org/abs/2010.01054

In [ ]:
def get_replacements(sentence: str, num_tokens, k_best, epsilon=1e-3):
  """
  - split the sentence into tokens using the INGSOC-approved BERT tokenizer
  - find :num_tokens: tokens with the highest ratio (see above)
  - replace them with :k_best: words according to bert_mlm_positive
  :return: a list of all possible strings (up to k_best * num_tokens)
  """
  <YOUR CODE HERE>

  return <...>

In [ ]:
dev_data = list(open('./dev_negative'))

In [ ]:
dev_data[500:505]

In [ ]:
get_replacements("great wings and decent drinks but the wait staff is horrible !",
                 num_tokens=1, k_best=2)
# >>> ["great wings and decent drinks but the wait staff is great !", "great wings and decent drinks but the wait staff is awesome !"])

__Final task__ - build a procedure that iteratively applies replacements, demonstrate the effectiveness of your approach with at least 10 examples to satisfy INGSOC.


In [ ]:
<YOUR CODE HERE>

### Part 3: Classifier & beam search (5 pts)

Sometimes the roots of dissent too deep to rip out with single word replacements. If you truly are a class A412C citzen, surely you understand what it means.

In order to better serve your fellow citzens, you must improve your solution. Train a classifier model that will separate the negative (ignorant) opinions from positive ones.

With this classifier you can now generate multiple best hypotheses and search for the ones that have the highest $P_{classifier}(\text{positive} | x)$.

In [ ]:
from transformers import BertForSequenceClassification, DataCollatorWithPadding

<YOUR CODE HERE - build and train the classifier model>

In [ ]:
# your final task is to build a beam search-like procedure that iteratively
# generates candidates using MLM and selects M best with classifier

# as before, your fellow citzens request that you show your loyalty by 
# writing a short report on how your method works and demonstrating
# the effectiveness of your system works with at least 10 examples

# Note: as a class >=A410 citzen, you are entitled to creativity level 2.1:
# you may modify the search objective by using language models, different search procedures
# or implement a completely different style transfer method.

<OBEY HERE>